In [1]:
#@title 安装依赖
!pip install torch diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets==7.7.1 huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
#@title huggingface 认证

from huggingface_hub import login
login()

In [4]:
#@title 选择模型
#导入必要的库和模块,包括 Diffusers、PyTorch 和 ipywidgets。
#定义交互式下拉菜单和复选框,用于选择模型、采样器和是否启用安全检查。
#定义 get_scheduler 函数,根据用户选择的采样器返回相应的调度器对象。
#定义 pipeline 函数,用于初始化 StableDiffusionPipeline。它会根据用户的选择加载指定的模型和采样器,并设置是否启用安全检查。此外,它还启用了 xformers 内存高效注意力机制。
#以上是AI回答

from diffusers import DiffusionPipeline, StableDiffusionPipeline, StableDiffusionXLPipeline, StableDiffusion3Pipeline, EulerAncestralDiscreteScheduler, DDIMScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Enable third party widget support
#from google.colab import output
#output.enable_custom_widget_manager()

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion XL Base 1.0", "stabilityai/stable-diffusion-xl-base-1.0"),
        ("Stable Diffusion 3 Midium", "stabilityai/stable-diffusion-3-medium-diffusers"),
        ("Stable Diffusion 2.1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2.1" , "stabilityai/stable-diffusion-2-1"),
        ("Stable Diffusion 1.5", "runwayml/stable-diffusion-v1-5"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4"),
        ("AnyLoRA Anime Mix - SD1.5", "adogecheems/aamAnyloraAnimeMixAnime_v1")
    ],
    description="Select Model:"
)

#Schedulers
select_sampler = widgets.Dropdown(
    options=[
        "EulerAncestralDiscreteScheduler",
        "EulerDiscreteScheduler",
        "UniPCMultistepScheduler",
        "DDIMScheduler"
    ],
    description="Select Schedular:"
)
select_sampler.style.description_width = "auto"

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Get scheduler
def get_scheduler(name):

  match name:

    case "EulerAncestralDiscreteScheduler":
      return EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "DDIMScheduler":
      return DDIMScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "EulerDiscreteScheduler":
      return EulerDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "UniPCMultistepScheduler":
      return UniPCMultistepScheduler.from_pretrained(select_model.value, subfolder="scheduler")

#Run pipeline
def pipeline(p):

  global pipe

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")
    
    if select_model.value == "stabilityai/stable-diffusion-3-medium-diffusers":
        # 加载 Stable Diffusion 3
        pipe = StableDiffusion3Pipeline.from_pretrained(
            select_model.value,
            scheduler=get_scheduler(select_sampler.value),
            torch_dtype=torch.float16).to("cuda")
        
    elif select_model.value == "stabilityai/stable-diffusion-xl-base-1.0":
        # 加载 Stable Diffusion XL
        pipe = StableDiffusionXLPipeline.from_pretrained(
            select_model.value,
            scheduler=get_scheduler(select_sampler.value),
            torch_dtype=torch.float16).to("cuda")
        
    else:
        # 加载其他低版本模型
        pipe = StableDiffusionPipeline.from_pretrained(
            select_model.value,
            scheduler=get_scheduler(select_sampler.value),
            torch_dtype=torch.float16).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, select_sampler, safety_check, apply_btn, out
    ]
)


In [5]:
#@title 生成 图片
#提示 - 图片描述
#负面提示- 您不想在图片中看到或忽略的内容
#步骤 - 去噪步骤数。步骤数越高，结果越好，但生成图片所需的时间越长。默认值为 `30`。
#CFG - 指导范围从 `0` 到 `20`。较低的值允许 AI 更具创造性，并且不太严格地遵循提示。默认值为 `7.5`。
#种子 - 控制图片生成的随机值。相同的种子和提示会产生相同的图片。设置 `-1` 表示使用随机种子值。

import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS
width = "300px"

prompt = widgets.Textarea(
    value="",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

neg_prompt = widgets.Textarea(
    value="",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=1,
    description="Images:",
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=30,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768), ('1024px', 1024)],
    value=512,
    description="Height:",
    layout=widgets.Layout(width=width)
)

img_width = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768), ('1024px', 1024)],
    value=512,
    description="Width:",
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  #Calculate seed
  seed = random.randint(0, 2147483647) if random_seed.value == -1 else random_seed.value

  with display_imgs:

    print("Running...")

    images = pipe(
      prompt.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(seed)
    ).images
    mediapy.show_images(images)

    print(f"Seed:\n{seed}")

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.HTML(
            value="<h2>Stable Diffusion</h2>",
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)

In [3]:
'''
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/kaggle/working/aamAnyloraAnimeMixAnime_v1",
    repo_id="adogecheems/aamAnyloraAnimeMixAnime_v1",
    repo_type="model",
)
'''

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adogecheems/aamAnyloraAnimeMixAnime_v1/commit/00e0c07492fa6edf1800c1518601160a21c5d59a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='00e0c07492fa6edf1800c1518601160a21c5d59a', pr_url=None, pr_revision=None, pr_num=None)